### Politicas basadas en recursos

Se trata de políticas insertadas asociadas a los recursos de AWS. Los ejemplos más comunes de políticas basadas en recursos son las políticas de bucket de Amazon S3 y las políticas de confianza de roles de IAM. Las políticas basadas en recursos conceden permisos a la entidad principal especificada en la política; por lo tanto, se requiere el elemento Principal en la política. 

* Conceden permiso a las entidades principales o cuentas (cuentas iguales o diferentes).

La política basada en recursos que aparece a continuación se asocia a un bucket de Amazon S3. Según la política, solo el usuario de IAM carlossalzar puede acceder a este bucket. 

In [ ]:
#pip install boto3 --upgrade

In [1]:
import boto3
import json
from botocore.exceptions import ClientError

In [2]:
s3 = boto3.client('s3')
# Create a bucket
bucket_name = 'my-bucket-test-494300'
try:
    s3.create_bucket(Bucket=bucket_name)
except ClientError as e:
    print(e)

In [19]:
policy_document_s3 = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "Example permissions",
            "Effect": "Allow", 
            "Action": "s3:*",
            "Principal": {
                "AWS": "arn:aws:iam::089715336747:user/carlossalzar"
            },
            "Resource": [
                f"arn:aws:s3:::{bucket_name}",
                f"arn:aws:s3:::{bucket_name}/*"
            ]
        }
    ]
}

In [20]:
# Put the policy to the bucket
try:
    s3.put_bucket_policy(
        Bucket=bucket_name,
        Policy=json.dumps(policy_document_s3)
    )
except ClientError as e:
    print(e)

### Limite de permisos

Un límite de permisos establece la cantidad máxima de permisos que una política basada en identidades puede conceder a una entidad de IAM. Esta entidad solo puede realizar las acciones que le permitan tanto sus políticas basadas en identidades como sus límites de permisos. Las políticas basadas en recursos que definen al usuario o rol como entidad principal no están limitadas por el límite de permisos.

* Restringe los permisos para la entidad de IAM asociada a ella.

Por ejemplo, supongamos que a uno de los usuarios de IAM se le debe permitir administrar solo Amazon S3, Amazon CloudWatch y Amazon EC2. Para aplicar esta regla, puede utilizar la política administrada por el cliente siguiente:

In [ ]:
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "s3:*",
                "ec2:*",
                "cloudwatch:*"
            ],
            "Resource": "*"
        }
    ]
}

Luego, agregue el bloque de condiciones que aparece a continuación a la política del usuario de IAM. El usuario nunca podrá realizar operaciones en ningún otro servicio, incluido IAM, aunque tenga una política de permisos que lo permita.

In [1]:
{
    "Condition": {
        "StringEquals": {"iam:PermissionsBoundary": "arn:aws:iam::089715336747:policy/test-policy"}
    }
}

{'Condition': {'StringEquals': {'iam:PermissionsBoundary': 'arn:aws:iam::089715336747:policy/test-policy'}}}

### Acesso a objetos en S3 basado en las etiquetas

Amazon S3 almacena datos en una estructura plana; se crea un bucket y el bucket almacena objetos. Amazon S3 no tiene una jerarquía de subcubetas o carpetas; sin embargo, herramientas como la consola de administración de AWS pueden emular una jerarquía de carpetas para presentar carpetas en un bucket utilizando los nombres de los objetos (también conocidos como claves). Para simplificar, puede pensar en el nombre de un objeto como la ruta completa de un archivo en un sistema de archivos tradicional.

Esta política concede el acceso completo de consola sólo a su carpeta (/home/DepartamentTagValue) y a la de nadie más. Aunque podría simplemente conceder a cada usuario acceso a su propio bucket, tenga en cuenta que una cuenta de AWS puede tener hasta 100 buckets por defecto. Mediante la creación de carpetas de inicio y la concesión de los permisos adecuados, puede hacer que cientos de usuarios compartan un único bucket.

In [ ]:
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "S3ConsoleAccess",
            "Effect": "Allow",
            "Action": [
                "s3:ListAllMyBuckets"
            ],
            "Resource": "*"
        },
        {
            "Sid": "ListObjectsInBucket",
            "Effect": "Allow",
            "Action": "s3:ListBucket",
            "Resource": "arn:aws:s3:::workshop-redshift-394-33",
            "Condition": {
                "StringLike": {
                    "s3:prefix": [
                        "",
                        "home/",
                        "home/${aws:PrincipalTag/Department}/*"
                    ]
                }
            }
        },
        {
            "Sid": "AllowAllS3ActionsInUserFolder",
            "Effect": "Allow",
            "Action": "s3:*",
            "Resource": [
                "arn:aws:s3:::workshop-redshift-394-33/home/${aws:PrincipalTag/Department}",
                "arn:aws:s3:::workshop-redshift-394-33/home/${aws:PrincipalTag/Department}/*"
            ]
        }
    ]
}